In [ ]:
import unicodedata
import re
import random
from datasets import load_dataset
from collections import defaultdict
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from collections import Counter

# linea que arregla algunos errores de loadeo de datasets
# pip install --upgrade datasets

/Users/malenaalamo/Personal/AA2/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Ejericio b)

### Imports

In [46]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from transformers import BertTokenizer, BertModel

# Descomentar en Windows
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Descomentar en Mac
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

### Definicion del DataLoader (importante)

In [ ]:
from transformers import BertTokenizerFast  # FAST tokenizer recomendado

# Cargo tokenizar
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

PUNCT_TAGS = {"Ø": 0, ",": 1, ".": 2, "?": 3, "¿": 4}
CAP_TAGS = {"lower": 0, "init": 1, "mix": 2, "upper": 3}

def _get_capitalization_type(word):
    if not word or word.islower(): return 0
    if word.istitle(): return 1
    if word.isupper(): return 3
    if any(c.isupper() for c in word[1:]): return 2
    return 0

def get_cap_labels_for_tokens(labels_per_word, token_word_map):
    """
    Recibe los labels por palabra y devuelve los labels por token para capitalizacion
    Si los subtokens pertenecen a la misma palabra, les pone el mismo label (capitalizacion) 
    """
    labels = []
    for word_idx in token_word_map:
        if word_idx is None:
            labels.append(-100)
        else:
            labels.append(labels_per_word[word_idx])
    return labels

def get_punct_labels_for_tokens(labels_per_word, token_word_map):
    """
    Asigna etiquetas de puntuación a los subtokens, siguiendo las reglas:
    - ¿ va en el primer subtoken de la palabra.
    - ., ?, , van en el último subtoken de la palabra.
    - Ø no se asigna a ningún subtoken (todos -100).
    """
    labels = [0] * len(token_word_map)
    word_to_token_idxs = {}

    # Construimos un diccionario: word_idx -> [lista de posiciones de tokens]
    for token_idx, word_idx in enumerate(token_word_map):
        if word_idx is not None:
            word_to_token_idxs.setdefault(word_idx, []).append(token_idx)

    for word_idx, token_idxs in word_to_token_idxs.items():
        punct_label = labels_per_word[word_idx]
        if punct_label == PUNCT_TAGS["¿"]:
            target_idx = token_idxs[0]  # primer subtoken
        elif punct_label in {PUNCT_TAGS["."], PUNCT_TAGS[","], PUNCT_TAGS["?"]}:
            target_idx = token_idxs[-1]  # último subtoken
        else:
            continue  # Ø: no se asigna nada

        labels[target_idx] = punct_label

    return labels


def get_dataloader(oraciones_raw, max_length, batch_size, device):
    """
    Crea un DataLoader para entrenar un modelo de restauración de puntuación y capitalización.

    A partir de una lista de oraciones correctamente escritas (con puntuación y mayúsculas),
    esta función:
        - Extrae etiquetas de puntuación y capitalización por palabra.
        - "Corrompe" el texto al eliminar la puntuación y poner las palabras en minúscula.
        - Tokeniza las palabras corruptas usando un tokenizer BERT.
        - Alinea las etiquetas con los subtokens del tokenizer.
        - Crea tensores para las entradas (input_ids, attention_mask) y etiquetas (puntuación y capitalización).
        - Devuelve un DataLoader para entrenamiento en lotes.

    Parámetros:
        oraciones_raw (List[str]): Lista de oraciones correctamente formateadas.
        max_length (int): Longitud máxima de secuencia para truncar/padear.
        batch_size (int): Tamaño del batch.
        device (str): Dispositivo donde se cargarán los tensores ('cpu' o 'cuda').

    Retorna:
        DataLoader: DataLoader que entrega batches de (input_ids, attention_mask, punct_labels, cap_labels).
    """
    input_ids_list = []
    attention_masks = []
    punct_labels_list = []
    cap_labels_list = []

    for sent in oraciones_raw:
        # Extraer palabras con puntuación
        matches = list(re.finditer(r"\b\w+[^\s\w]?\b", sent)) # Detecta puntuaciones y las splitea
        words = []
        punct_labels = []
        cap_labels = []

        for i, m in enumerate(matches): # Recorre cada palabra detectada
            word_raw = m.group(0) 
            clean_word = re.sub(r"[.,?¿]", "", word_raw) # Limpia la palabra "Hola!" -> "Hola"

            # Puntuación
            before = sent[m.start() - 1] if m.start() > 0 else "" # Signo anterior
            after = sent[m.end()] if m.end() < len(sent) else ""  # Signo posterior
            if before == '¿':
                punct = PUNCT_TAGS["¿"]
            elif after in PUNCT_TAGS:
                punct = PUNCT_TAGS[after]
            else:
                punct = PUNCT_TAGS["Ø"]

            # Capitalización
            cap = _get_capitalization_type(word_raw)

            clean_word = clean_word.lower() # Limpia la palabra Hola -> hola

            words.append(clean_word)
            punct_labels.append(punct)
            cap_labels.append(cap)

        # Tokenización con BERT
        encoding = tokenizer(words,
                             is_split_into_words=True,
                             return_tensors='pt',
                             padding='max_length',
                             truncation=True,
                             max_length=max_length,
                             return_attention_mask=True)

        # Extraer datos que nos sirven del encoding
        input_ids = encoding['input_ids'][0]
        attention_mask = encoding['attention_mask'][0]
        word_ids = encoding.word_ids(batch_index=0)  # Mapea cada subtoken a su palabra

        # Alinear etiquetas a subtokens (hasta ahora las teniamos en palabras)
        punct_labels_aligned = get_punct_labels_for_tokens(punct_labels, word_ids)
        cap_labels_aligned = get_cap_labels_for_tokens(cap_labels, word_ids)

        # Convertir a tensores
        punct_tensor = torch.tensor(punct_labels_aligned)
        cap_tensor = torch.tensor(cap_labels_aligned)

        # Aplicar -100 a posiciones de padding
        punct_tensor[attention_mask == 0] = -100
        cap_tensor[attention_mask == 0] = -100

        # Agregar a listas (por oracion)
        input_ids_list.append(input_ids)
        attention_masks.append(attention_mask)
        punct_labels_list.append(punct_tensor)
        cap_labels_list.append(cap_tensor)

    # Stackear tensores (por batch)
    input_ids = torch.stack(input_ids_list).to(device)
    attention_masks = torch.stack(attention_masks).to(device)
    punct_labels = torch.stack(punct_labels_list).to(device)
    cap_labels = torch.stack(cap_labels_list).to(device)

    dataset = TensorDataset(input_ids, attention_masks, punct_labels, cap_labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return dataloader


In [142]:
# Linea para entender como el get loader determina las puntuaciones

import re

# Ejemplo de oración
sentence = "¿Te gusta la soda? Que raro, a mi me gusta mas tomar CocaCola"

# Encuentra palabras con posible puntuación al final
matches = list(re.finditer(r"\b\w+[^\s\w]?\b", sentence))

# Mostramos los resultados
for match in matches:
    word_raw = match.group(0)
    start = match.start()
    end = match.end()

    # Caracter anterior y posterior
    before = sentence[start - 1] if start > 0 else ""
    after = sentence[end] if end < len(sentence) else ""

    print(f"Palabra detectada : '{word_raw}'")
    print(f"Antes del match   : '{before}'")
    print(f"Después del match : '{after}'")
    print("---")


Palabra detectada : 'Te'
Antes del match   : '¿'
Después del match : ' '
---
Palabra detectada : 'gusta'
Antes del match   : ' '
Después del match : ' '
---
Palabra detectada : 'la'
Antes del match   : ' '
Después del match : ' '
---
Palabra detectada : 'soda'
Antes del match   : ' '
Después del match : '?'
---
Palabra detectada : 'Que'
Antes del match   : ' '
Después del match : ' '
---
Palabra detectada : 'raro'
Antes del match   : ' '
Después del match : ','
---
Palabra detectada : 'a'
Antes del match   : ' '
Después del match : ' '
---
Palabra detectada : 'mi'
Antes del match   : ' '
Después del match : ' '
---
Palabra detectada : 'me'
Antes del match   : ' '
Después del match : ' '
---
Palabra detectada : 'gusta'
Antes del match   : ' '
Después del match : ' '
---
Palabra detectada : 'mas'
Antes del match   : ' '
Después del match : ' '
---
Palabra detectada : 'tomar'
Antes del match   : ' '
Después del match : ' '
---
Palabra detectada : 'CocaCola'
Antes del match   : ' '
Después

### Dataset

In [238]:
# Armado del dataset

DATA_URLS = {
    "train": "https://huggingface.co/datasets/PlanTL-GOB-ES/SQAC/resolve/main/train.json",
    "dev":   "https://huggingface.co/datasets/PlanTL-GOB-ES/SQAC/resolve/main/dev.json",
    "test":  "https://huggingface.co/datasets/PlanTL-GOB-ES/SQAC/resolve/main/test.json",
}

raw = load_dataset(
    "json",
    data_files=DATA_URLS,
    field="data",
)

questions = []


for i in range(0, len(raw["train"])):
  for p in raw["train"][i]['paragraphs']:
    p_questions = [qas['question'] for qas in p['qas']]
    questions += p_questions

N_QUESTIONS = 5000
questions = questions[:N_QUESTIONS]
print(f"Se descargaron {len(questions)} preguntas en Español.")


Se descargaron 5000 preguntas en Español.


In [239]:
dataset_rnn = load_dataset("google/wmt24pp", "en-es_MX", split="train")
oraciones_rnn = dataset_rnn['target'][1:]

print(f"Se descargaron {len(oraciones_rnn)} oraciones en Español (del dataset del notebook 10).")

Se descargaron 997 oraciones en Español (del dataset del notebook 10).


In [240]:
oraciones_sinteticas = []
import json
with open('./datasets.json', 'r') as file:
  data = json.load(file)

oraciones_sinteticas = data['otros'] + data['marcas']
print(f"Hay {len(oraciones_sinteticas)} oraciones sintéticas.")

Hay 1413 oraciones sintéticas.


In [268]:
import wikipedia

# API Wikipedia
wikipedia.set_lang("es")

def obtener_frases_wikipedia(titulo, max_frases=100):
    try:
        pagina = wikipedia.page(titulo)
        texto = pagina.content
        oraciones = re.split(r'(?<=[.!?])\s+', texto)
        frases = [o.strip() for o in oraciones if 5 < len(o.split()) < 30]
        return frases[:max_frases]
    except Exception as e:
        print(f"Error al buscar '{titulo}': {e}")
        return []

# Ejemplo: obtener 50 frases de un artículo
frases = obtener_frases_wikipedia("Revolución francesa", max_frases=50)
for f in frases[:5]:
    print(f"- {f}")

temas = [
    # Países y lugares
    'Argentina', 'España', 'México', 'Colombia', 'Chile',
    'Perú', 'Uruguay', 'Brasil', 'América Latina', 'Europa',

    # Cultura argentina
    'Lionel Messi', 'Diego Maradona', 'Lali Esposito', 'Charly Garcia', 'Dillom',
    'Tiempos Violentos', 'Relatos Salvajes', 'Universidad de Buenos Aires', 'Rock nacional', 'Cine argentino',

    # Historia y política
    'Revolucion de Mayo', 'Independencia de Argentina', 'Simón Bolívar', 'Segunda Guerra Mundial', 'Guerra Fría',
    'Revolución Francesa', 'Guerra Civil Española', 'Napoleón Bonaparte', 'Nelson Mandela', 'Dictadura militar en Argentina',

    # Ciencia y tecnología
    'Inteligencia artificial', 'ChatGPT', 'Redes neuronales', 'Robótica', 'Energía solar',
    'Vacunas', 'COVID-19', 'Cambio climático', 'Computadora cuántica', 'NASA',

    # Cultura general
    'El Principito', 'Premio Nobel', 'Frida Kahlo', 'Pablo Picasso', 'Leonardo da Vinci',
    'William Shakespeare', 'Gabriel García Márquez', 'Julio Cortázar', 'Literatura latinoamericana', 'Arte contemporáneo',

    # Entretenimiento y medios
    'Marvel', 'DC Comics', 'Netflix', 'Cine de terror', 'Películas de ciencia ficción',
    'Música electrónica', 'Reguetón', 'Spotify', 'YouTube', 'TikTok',

    # Deportes
    'Fútbol', 'Copa Mundial de la FIFA', 'Juegos Olimpicos', 'Tenis', 'NBA',
    'Boca Juniors', 'River Plate', 'Messi vs Ronaldo', 'Fórmula 1', 'Michael Jordan',

    # Sociedad y actualidad
    'Feminismo', 'Día Internacional de la Mujer', 'Diversidad cultural', 'Migración', 'Pobreza',
    'Educación pública', 'Salud mental', 'Medio ambiente', 'Derechos humanos', 'Trabajo remoto',

    # Filosofía y pensamiento
    'Filosofía', 'Ética', 'Psicología', 'Sigmund Freud', 'Carl Jung',
    'Existencialismo', 'Sociología', 'Economía', 'Política', 'Democracia'
]


frases_wikipedia = []
for tema in temas:
    print(f"Obteniendo frases de Wikipedia para: {tema}")
    frases = obtener_frases_wikipedia(tema,max_frases=100)
    print('Ejemplos de frases obtenidas:')
    for f in frases[:2]:
        print(f"- {f}")
    frases_wikipedia.extend(frases)

- La corriente de pensamiento vigente en Francia era la Ilustración, cuyos principios se basaban en la razón, la igualdad y la libertad.
- La Ilustración había servido de impulso a las Trece Colonias norteamericanas para la independencia de su metrópolis europea.
- Tanto la influencia de la Ilustración como el ejemplo de los Estados Unidos sirvieron de «trampolín» ideológico para el inicio de la revolución en Francia.
- El otro gran lastre para la economía fue la deuda estatal.
- En 1788, la relación entre la deuda y la renta nacional bruta en Francia era del 55,6 %, en comparación con el 181,8 % en Gran Bretaña.
Obteniendo frases de Wikipedia para: Argentina
Ejemplos de frases obtenidas:
- Argentina, oficialmente República Argentina,[a]​ es un país soberano de América del Sur, ubicado en el extremo sur y sudeste de ese subcontinente.
- Adopta la forma de gobierno republicana, democrática, representativa y federal.
Obteniendo frases de Wikipedia para: España
Ejemplos de frases obtenida

/Users/malenaalamo/Personal/AA2/venv/lib/python3.9/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/malenaalamo/Personal/AA2/venv/lib/python3.9/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Error al buscar 'Redes neuronales': "Red neuronal" may refer to: 
red neuronal biológica
red neuronal artificial
Ejemplos de frases obtenidas:
Obteniendo frases de Wikipedia para: Robótica
Ejemplos de frases obtenidas:
- La robótica es la disciplina que se ocupa del diseño, operación, manufacturación, estudio y aplicación de autómatas o robots.
- (Robots Universales Rossum), escrita por Karel Čapek en 1920.
Obteniendo frases de Wikipedia para: Energía solar
Ejemplos de frases obtenidas:
- La energía solar es una energía renovable, obtenida a partir del aprovechamiento de la radiación electromagnética procedente del Sol.
- La radiación solar que alcanza la Tierra ha sido aprovechada por el ser humano desde la antigüedad, mediante diferentes tecnologías que han ido evolucionando.
Obteniendo frases de Wikipedia para: Vacunas
Ejemplos de frases obtenidas:
- Existen cuatro tipos de vacunas principales:[10]​

Vivas atenuadas: microorganismos que han sido cultivados expresamente bajo condicio

In [271]:
len(frases_wikipedia)

6648

In [290]:
oraciones_raw = questions + oraciones_rnn + oraciones_sinteticas + frases_wikipedia

print(len(oraciones_raw))

import random

random.sample(oraciones_raw, 30)

14058


['¿Cuál es el cargo actual de Ilques Barbosa Júnior?',
 'Woolf, Virginia (1999, edición original 1938).',
 'El sistema ha de ser escalable: tiene que haber una forma definida de aumentar el número de cúbits, para tratar con problemas de mayor coste computacional.',
 'En 1894 Japón, que ya hacía tiempo que se disputaba la península de Corea con el Imperio Chino, inició la Primera Guerra Sino-japonesa con un ataque sin previo aviso.',
 '¿Cuándo se declaró la ley de emergencia pesquera?',
 '¿Cómo se define la actitud de Becket respecto a la tradición realista?',
 '¿Qué empresa fue la encargada de fabricar esta aeronave?',
 'Realizó varios retratos de Elvira Paladini y de Apollinaire, en un estilo realista, y varios dibujos de arlequines que recuerdan en su trazo a la época rosa.',
 'Para lograrlo, ACE se basa en seis áreas clave: educación, sensibilización, capacitación, participación pública, acceso a la información y cooperación.',
 'El enemigo de mi enemigo es mi amigo. ¡No estoy de ac

In [315]:
from sklearn.model_selection import train_test_split

train_sents, test_sents = train_test_split(oraciones_raw, test_size=0.05, random_state=42)

dataloader_train = get_dataloader(oraciones_raw=oraciones_raw, max_length=64, batch_size=64, device=device)
dataloader_test = get_dataloader(oraciones_raw=test_sents, max_length=64, batch_size=64, device=device)

print(len(train_sents))
print(len(test_sents))

13355
703


### RNN

In [316]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


class PunctuationCapitalizationRNN(nn.Module):
    def __init__(self, bert_embeddings, hidden_dim, num_punct_classes, num_cap_classes, dropout=0.3):
        super().__init__()
        self.embedding = bert_embeddings  # Embeddings de BERT

        # Capa de proyección: adapta dimensión de salida de BERT al hidden_dim de la RNN
        self.projection = nn.Linear(self.embedding.embedding_dim, hidden_dim)

        # LSTM más profunda: 2 capas
        self.rnn = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=2,
            batch_first=True,
            bidirectional=False  # manteniendo unidireccional
        )

        self.dropout = nn.Dropout(dropout)

        # Clasificador de puntuación con capa oculta
        self.punct_classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_punct_classes)
        )

        # Clasificador de capitalización con capa oculta
        self.cap_classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_cap_classes)
        )

    def forward(self, input_ids, attention_mask=None):
        # Embeddings de BERT (sin fine-tuning)
        with torch.no_grad():
            embedded = self.embedding(input_ids)  # (batch, seq_len, emb_dim)
            embedded = embedded.detach()

        # Proyección a hidden_dim
        projected = self.projection(embedded)  # (batch, seq_len, hidden_dim)

        # LSTM con manejo de padding
        if attention_mask is not None:
            lengths = attention_mask.sum(dim=1)
            packed = pack_padded_sequence(projected, lengths.cpu(), batch_first=True, enforce_sorted=False)
            rnn_out_packed, _ = self.rnn(packed)
            rnn_out, _ = pad_packed_sequence(rnn_out_packed, batch_first=True, total_length=projected.size(1))
        else:
            rnn_out, _ = self.rnn(projected)

        # Regularización
        rnn_out = self.dropout(rnn_out)

        # Cabezales separados
        punct_logits = self.punct_classifier(rnn_out)
        cap_logits = self.cap_classifier(rnn_out)

        return punct_logits, cap_logits

In [317]:
# Funcion de entrenamiento
def train(model, dataloader_train, dataloader_test, optimizer, criterion, device, epochs=3):
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for input_ids, attention_mask, punct_labels, cap_labels in dataloader_train:
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            punct_labels = punct_labels.to(device)
            cap_labels = cap_labels.to(device)

            optimizer.zero_grad()

            punct_logits, cap_logits = model(input_ids, attention_mask)

            loss_punct = criterion(punct_logits.view(-1, punct_logits.shape[-1]), punct_labels.view(-1))
            loss_cap = criterion(cap_logits.view(-1, cap_logits.shape[-1]), cap_labels.view(-1))

            loss = loss_punct + loss_cap
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(dataloader_train)

        """
        # Evaluación en test
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for input_ids, attention_mask, punct_labels, cap_labels in dataloader_test:
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                punct_labels = punct_labels.to(device)
                cap_labels = cap_labels.to(device)

                punct_logits, cap_logits = model(input_ids, attention_mask)

                loss_punct = criterion(punct_logits.view(-1, punct_logits.shape[-1]), punct_labels.view(-1))
                loss_cap = criterion(cap_logits.view(-1, cap_logits.shape[-1]), cap_labels.view(-1))

                loss = loss_punct + loss_cap
                test_loss += loss.item()

        avg_test_loss = test_loss / len(dataloader_test)
        """

        print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f}")



# Funcion de evaluacion
def evaluate(model, dataloader, device):
    model.eval()
    total_punct_correct = 0
    total_cap_correct = 0
    total_tokens = 0

    with torch.no_grad():
        for input_ids, attention_mask, punct_labels, cap_labels in dataloader:
            input_ids = input_ids.to(device)
            punct_labels = punct_labels.to(device)
            cap_labels = cap_labels.to(device)

            punct_logits, cap_logits = model(input_ids)

            # Obtener predicciones (dim: [batch, seq_len])
            pred_punct = torch.argmax(punct_logits, dim=-1)
            pred_cap = torch.argmax(cap_logits, dim=-1)

            # Máscara válida (para ignorar -100)
            mask = (punct_labels != -100)

            # Cálculo de accuracy
            total_punct_correct += (pred_punct[mask] == punct_labels[mask]).sum().item()
            total_cap_correct += (pred_cap[mask] == cap_labels[mask]).sum().item()
            total_tokens += mask.sum().item()

    punct_acc = total_punct_correct / total_tokens
    cap_acc = total_cap_correct / total_tokens

    print(f"📌 Punctuation Accuracy:     {punct_acc:.4f}")
    print(f"🔡 Capitalization Accuracy: {cap_acc:.4f}")
    return punct_acc, cap_acc


In [318]:
# Embeddings de BERT

model_name = "bert-base-multilingual-cased"
bert_model = BertModel.from_pretrained(model_name)

# Extraemos la capa de embeddings de BERT
bert_embeddings = bert_model.embeddings.word_embeddings  # Es una instancia de nn.Embedding

# (Opcional) Congelamos los embeddings para que no se modifiquen durante el entrenamiento
for param in bert_embeddings.parameters():
    param.requires_grad = False


# Crear el modelo
model = PunctuationCapitalizationRNN(
    bert_embeddings=bert_embeddings,
    hidden_dim=256,
    num_punct_classes=len(PUNCT_TAGS),
    num_cap_classes=len(CAP_TAGS)
).to(device)

# Configurar optimizador y función de pérdida
criterion = nn.CrossEntropyLoss(ignore_index=-100)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay=0.01)

# Entrenamiento 
train(model, dataloader_train=dataloader_train, dataloader_test=dataloader_test,optimizer=optimizer, criterion=criterion, device=device, epochs=20)

Epoch 1 | Train Loss: 1.1642
Epoch 2 | Train Loss: 0.6984
Epoch 3 | Train Loss: 0.6206
Epoch 4 | Train Loss: 0.5602
Epoch 5 | Train Loss: 0.5203
Epoch 6 | Train Loss: 0.5026
Epoch 7 | Train Loss: 0.4882
Epoch 8 | Train Loss: 0.4714
Epoch 9 | Train Loss: 0.4545
Epoch 10 | Train Loss: 0.4412
Epoch 11 | Train Loss: 0.4307
Epoch 12 | Train Loss: 0.4228
Epoch 13 | Train Loss: 0.4171
Epoch 14 | Train Loss: 0.4100
Epoch 15 | Train Loss: 0.4037
Epoch 16 | Train Loss: 0.3987
Epoch 17 | Train Loss: 0.3931
Epoch 18 | Train Loss: 0.3910
Epoch 19 | Train Loss: 0.3863
Epoch 20 | Train Loss: 0.3808


In [319]:
evaluate(model, dataloader_test, device)

📌 Punctuation Accuracy:     0.9274
🔡 Capitalization Accuracy: 0.8566


(0.9274139992030814, 0.8565546553327135)

In [303]:
def predict_and_reconstruct(model, sentence, tokenizer, device, max_length=64):
    model.eval()

    encoding = tokenizer(
        sentence,
        return_tensors='pt',
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_attention_mask=True,
        return_token_type_ids=False
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        punct_logits, cap_logits = model(input_ids, attention_mask=attention_mask)

    pred_punct = torch.argmax(punct_logits, dim=-1)[0].cpu().tolist()
    pred_cap = torch.argmax(cap_logits, dim=-1)[0].cpu().tolist()
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    INV_PUNCT_TAGS = {v: k for k, v in PUNCT_TAGS.items()}

    final_words = []
    current_word = ""
    current_cap = 0
    current_punct = 0
    new_word = True

    print("\n🔍 Predicción token por token:")
    print(f"{'TOKEN':15s} | {'PUNCT':>5s} | {'SIGNO':>5s} | {'CAP':>3s} | {'FINAL':15s}")
    print("-" * 55)

    for i, (token, punct_label, cap_label) in enumerate(zip(tokens, pred_punct, pred_cap)):
        if token in ["[CLS]", "[SEP]", "[PAD]"] or attention_mask[0, i].item() == 0:
            continue

        clean_token = token.replace("##", "")

        if token.startswith("##"):
            current_word += clean_token
            if punct_label != 0:
                current_punct = punct_label  # usar puntuación del último subtoken relevante
        else:
            if current_word:
                # cerrar palabra anterior
                word = current_word
                # aplicar capitalización a toda la palabra
                if current_cap == 1:
                    word = word.capitalize()
                elif current_cap == 2:
                    word = ''.join(c.upper() if random.random() > 0.5 else c.lower() for c in word)
                elif current_cap == 3:
                    word = word.upper()
                # aplicar puntuación del último subtoken
                punct = INV_PUNCT_TAGS.get(current_punct, "Ø")
                if punct == "¿":
                    word = "¿" + word
                elif punct != "Ø":
                    word = word + punct
                final_words.append(word)

            # empezar nueva palabra
            current_word = clean_token
            current_cap = cap_label
            current_punct = punct_label if punct_label != 0 else 0


        print(f"{clean_token:15s} | {punct_label:5d} | {INV_PUNCT_TAGS.get(punct_label, 'Ø'):>5s} | {cap_label:3d} | {clean_token:15s}")

    # Procesar última palabra
    if current_word:
        word = current_word
        if current_cap == 1:
            word = word.capitalize()
        elif current_cap == 2:
            word = ''.join(c.upper() if random.random() > 0.5 else c.lower() for c in word)
        elif current_cap == 3:
            word = word.upper()
        punct = INV_PUNCT_TAGS.get(current_punct, "Ø")
        if punct == "¿":
            word = "¿" + word
        elif punct != "Ø":
            word = word + punct
        final_words.append(word)

    return " ".join(final_words)


In [330]:
entrada = "me invito a comer jorge vos queres venir o le digo que no"
print("Predicción:", predict_and_reconstruct(model, entrada, tokenizer, device))



🔍 Predicción token por token:
TOKEN           | PUNCT | SIGNO | CAP | FINAL          
-------------------------------------------------------
me              |     0 |     Ø |   1 | me             
in              |     0 |     Ø |   0 | in             
vito            |     0 |     Ø |   0 | vito           
a               |     0 |     Ø |   0 | a              
comer           |     0 |     Ø |   0 | comer          
jo              |     0 |     Ø |   1 | jo             
rge             |     0 |     Ø |   1 | rge            
vos             |     0 |     Ø |   1 | vos            
quer            |     0 |     Ø |   0 | quer           
es              |     0 |     Ø |   0 | es             
venir           |     0 |     Ø |   0 | venir          
o               |     0 |     Ø |   0 | o              
le              |     0 |     Ø |   0 | le             
dig             |     0 |     Ø |   0 | dig            
o               |     0 |     Ø |   0 | o              
que             |

### Prueba con overfitting

In [314]:
frases = ["Buenas tardes, quiero un APPLE por favor. Muchisimas HH"]

train_loader = get_dataloader(frases, max_length=25, batch_size=1, device=device)

model = PunctuationCapitalizationRNN(
    bert_embeddings=bert_embeddings,
    hidden_dim=64,
    num_punct_classes=5,
    num_cap_classes=4
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)  # Alto LR
criterion = nn.CrossEntropyLoss(ignore_index=-100)

train(model, train_loader, train_loader,optimizer, criterion, device, epochs=200)

entrada = "buenas tardes quiero un apple por favor muchisimas hh"
print("Predicción:", predict_and_reconstruct(model, entrada, tokenizer, device))


Epoch 1 | Train Loss: 2.9770
Epoch 2 | Train Loss: 2.8499
Epoch 3 | Train Loss: 2.5912
Epoch 4 | Train Loss: 2.0590
Epoch 5 | Train Loss: 1.8295
Epoch 6 | Train Loss: 1.9136
Epoch 7 | Train Loss: 1.8124
Epoch 8 | Train Loss: 1.6952
Epoch 9 | Train Loss: 1.7456
Epoch 10 | Train Loss: 1.5041
Epoch 11 | Train Loss: 1.5334
Epoch 12 | Train Loss: 1.6429
Epoch 13 | Train Loss: 1.5945
Epoch 14 | Train Loss: 1.7140
Epoch 15 | Train Loss: 1.6594
Epoch 16 | Train Loss: 1.5866
Epoch 17 | Train Loss: 1.6796
Epoch 18 | Train Loss: 1.6025
Epoch 19 | Train Loss: 1.6516
Epoch 20 | Train Loss: 1.5347
Epoch 21 | Train Loss: 1.5403
Epoch 22 | Train Loss: 1.3112
Epoch 23 | Train Loss: 1.3635
Epoch 24 | Train Loss: 1.3845
Epoch 25 | Train Loss: 1.2871
Epoch 26 | Train Loss: 1.4506
Epoch 27 | Train Loss: 1.1565
Epoch 28 | Train Loss: 1.2059
Epoch 29 | Train Loss: 1.1263
Epoch 30 | Train Loss: 1.1023
Epoch 31 | Train Loss: 1.1991
Epoch 32 | Train Loss: 1.0047
Epoch 33 | Train Loss: 0.8669
Epoch 34 | Train Lo

### Export modelo

In [327]:
torch.save(model, "modelo.pt")